In [8]:
import networkx as nx
import numpy as np
import pickle 


In [6]:
data = 'environment'
path = "../gs_taxo/EN/" + str(data) + "_eurovoc_en.taxo"
G = nx.DiGraph()

with open(path, "r") as f:
    for line in f:
        idx, hypo, hyper = line.split("\t")
        hyper = hyper.replace("\n", "")
        G.add_node(hypo)
        G.add_node(hyper)
        G.add_edge(hyper, hypo)

root = data
all_verteces = list(G.nodes)
all_verteces.remove(root)

In [9]:
def clean_dict(pairs, use_lemma, reverse):
    new_pairs = {}
    for key, val in pairs.items():
        if use_lemma:
            term = key[0].split("(")[0].strip()
        else:
            term = key[0]
        target = key[1].split(",")[0]
        new_key = (target, term) if reverse else (term, target)
        new_pairs[new_key] = val

    return new_pairs

in_name = '../data/env/eurovoc/pred_hypernym/lemmas_ppl_clear.pickle'
lemma = True
reverse = True
with open(in_name, "rb") as f:
    ppls = pickle.load(f)

ppls_pairs = clean_dict(ppls, use_lemma=lemma, reverse=reverse)

In [25]:
def get_graph(G, ppl_pairs):
    nodes = list(G.nodes())
    S = nx.DiGraph()
    for node in nodes:
        candidates = {}
        for key, val in ppl_pairs.items():
            u, v = key
            if u == node:
                candidates[v] = val

        best_node, best_val = min(candidates.items(), key=lambda x: x[1])
        S.add_edge(node, best_node, weight=best_val)
    return S

In [26]:
G_pred = get_graph(G, ppls_pairs)

In [27]:
P = len(set(G.edges()) & set(G_pred.edges())) / (
    len(set(G_pred.edges())) + 1e-15
)
R = len(set(G.edges()) & set(G_pred.edges())) / len(set(G.edges()))
F = (2 * P * R) / (P + R + 1e-15)

print(P, R, F)

0.12643678160919541 0.12643678160919541 0.12643678160919491


In [20]:
len(set(G.edges()))

261